In [66]:
jovian.commit()

[jovian] Saving notebook..


<IPython.core.display.Javascript object>

[jovian] Updating notebook "b8a4a96727ae413a9c9c48bed4176d02" on https://jovian.ml/
[jovian] Uploading notebook..
[jovian] Capturing environment..
[jovian] Committed successfully! https://jovian.ml/thek9304/b8a4a96727ae413a9c9c48bed4176d02


# 뒤 3분

In [13]:
import csv
import glob
import jovian
import numpy
import pandas as pd

root_path = 'E:/eye_datacheck/raw_data/'
csv_save_path = './result4/'

csv_paths = glob.glob(root_path + '*.csv')
csv_paths = csv_paths[:-1]

csv_column = ['ori_Area', 'nom_Area', 'change_rate', 
              'Center_x', 'Center_y', 'Blink', 
              'Frame_num', 't_avg_size', 't_aver_change', 'c_aver_change', 'r_aver_change', 'Blink_cnt', 'Close_cnt', 'Blink_rate', 'Close_rate']
            # contraction, relaxation
csv_column_total = ['file_name', 'total_frame', 't_avg_size', 't_avg_change', 'c_avg_change',
                   'r_avg_change', 'blink_cnt', 'close_cnt', 'blink_rate', 'close_rate']
    
s_csv_list_total = []

# std, gamma, temp, bright
bright_t_avg_size = [[], [], [], []]
bright_t_avg_change = [[], [], [], []]
bright_blink_rate = [[], [], [], []]
bright_close_rate = [[], [], [], []]
dark_t_avg_size = [[], [], [], []]
dark_t_avg_change = [[], [], [], []]
dark_blink_rate = [[], [], [], []]
dark_close_rate = [[], [], [], []]

for csv_path in csv_paths:
    file_name = csv_path.split('\\')[-1][:-4]
    env_name = csv_path.split('\\')[-1].split('_')[1]
    condition = csv_path.split('\\')[-1].split('_')[2]
    
    csv = pd.read_csv(csv_path, error_bad_lines=False)
    ori_csv = list(csv['Area'])
    
    if file_name.split('_')[-1] == '1':
        mean_bright = 0
        for idx in range(csv.shape[0]):
            mean_bright += csv['Area'][idx]
        mean_bright /= csv.shape[0]
    else:
        std_idx = csv.shape[0] - 5401
        last_idx = csv.shape[0] - 1
#         std_idx = 0
#         last_idx = 5400
        
        s_csv_list = []
        t_avg_size = 0
        csv['Area'] -= mean_bright
        
        blink = False
        blink_cnt = 0
        close_cnt = 0
        bf_lists = []
        m_change = []
        p_change = []
        change_cnt = 0
        total_change = 0
        
        ff_cnt = 0
        for idx in range(std_idx, last_idx):
            t_avg_size += ori_csv[idx]
            
            c_x = csv['Center'][idx].strip('()').split(',')[0]
            c_y = csv['Center'][idx].strip('()').split(',')[1].lstrip()
             
            change_rate = ''
            if blink == False:
                if csv['Area'][idx] == -mean_bright and idx != std_idx:
                    blink = True
                    bf_list = [idx]
                    blink_start_idx = idx
                    blink_cnt += 1  
                    
                    before_area = csv['Area'][idx - 1]
        
                derivative = ori_csv[idx + 1] - ori_csv[idx]
                
                if idx == std_idx:
                    s_idx = idx
                    if derivative >= 0:
                        sign_check = True
                    else:
                        sign_check = False
                else:
                    change_f = -1
                    change_a = 0
                    if sign_check == True:
                        if derivative < 0:
                            change_f = idx - s_idx
                            change_a = ori_csv[idx] - ori_csv[s_idx]
                            sign_check = False
                            s_idx = idx
                            
                    else:
                        if derivative > 0:
                            change_f = idx - s_idx
                            change_a = ori_csv[idx] - ori_csv[s_idx]
                            sign_check = True
                            s_idx = idx
                    
                    if change_f != -1:
                        change_cnt += 1
                        
                        change_rate = change_a / change_f
                        
                        total_change += abs(change_rate)
                        
                        if change_rate > 0 :
                            p_change.append(change_rate)
                        else:
                            m_change.append(change_rate)
                
                if blink == True:
                    row = [ori_csv[idx], csv['Area'][idx], change_rate, c_x, c_y, int(blink)]
                else:
                    row = [ori_csv[idx], csv['Area'][idx], change_rate, c_x, c_y, int(blink)]
            else:
                derivative = 0
                
                row = [ori_csv[idx], csv['Area'][idx], change_rate, c_x, c_y, int(blink)]
            s_csv_list.append(row)
            
            if blink == True:
                close_cnt += 1
                if idx < last_idx - 1:
                    diff = csv['Area'][idx+1] - csv['Area'][idx] 
                    if diff < 0:
                        blink = False
                        bf_list.append(idx)
                        bf_lists.append(bf_list)
                        
                if idx == last_idx - 1:
                    bf_list.append(idx)
                    bf_lists.append(bf_list)
                    
            ff_cnt += 1
        
        s_csv_list[0].append(last_idx-std_idx)
        s_csv_list[0].append(t_avg_size / (last_idx -1- std_idx))
        s_csv_list[0].append(total_change / change_cnt)
        s_csv_list[0].append(sum(m_change) / len(m_change))
        s_csv_list[0].append(sum(p_change) / len(p_change))
        s_csv_list[0].append(blink_cnt)
        s_csv_list[0].append(close_cnt)
        s_csv_list[0].append(blink_cnt / ((last_idx - std_idx)/30))
        s_csv_list[0].append(close_cnt / ((last_idx - std_idx)/30))
        s_csv = pd.DataFrame(s_csv_list, columns=csv_column)
        csv_name = csv_save_path + file_name + '.csv'
        s_csv.to_csv(csv_name)
        
        file_name = csv_path.split('\\')[-1][:-7]
        s_csv_list_total.append(
                            [file_name, last_idx - std_idx, t_avg_size / (last_idx-1-std_idx), total_change / change_cnt,
                           sum(m_change) / len(m_change), sum(p_change) / len(p_change),
                            blink_cnt, close_cnt, blink_cnt / ((last_idx- std_idx)/30), close_cnt / ((last_idx- std_idx)/30)]
                            )
        
        # std, gamma, temp, bright
        if env_name == 'bright':
            if condition == 'std':
                bright_t_avg_size[0].append(t_avg_size / (last_idx - std_idx))
                bright_t_avg_change[0].append(total_change / change_cnt)
                bright_blink_rate[0].append(blink_cnt / ((last_idx - std_idx)/30))
                bright_close_rate[0].append(close_cnt / ((last_idx - std_idx)/30))
            elif condition == 'gamma':
                bright_t_avg_size[1].append(t_avg_size / (last_idx-1-std_idx))
                bright_t_avg_change[1].append(total_change / change_cnt)
                bright_blink_rate[1].append(blink_cnt / ((last_idx- std_idx)/30))
                bright_close_rate[1].append(close_cnt / ((last_idx- std_idx)/30))
            elif condition == 'temp':
                bright_t_avg_size[2].append(t_avg_size / (last_idx-1-std_idx))
                bright_t_avg_change[2].append(total_change / change_cnt)
                bright_blink_rate[2].append(blink_cnt / ((last_idx- std_idx)/30))
                bright_close_rate[2].append(close_cnt / ((last_idx- std_idx)/30))
            elif condition == 'bright':
                bright_t_avg_size[3].append(t_avg_size / (last_idx-1-std_idx))
                bright_t_avg_change[3].append(total_change / change_cnt)
                bright_blink_rate[3].append(blink_cnt / ((last_idx- std_idx)/30))
                bright_close_rate[3].append(close_cnt / ((last_idx- std_idx)/30))

        elif env_name == 'dark':
            if condition == 'std':
                dark_t_avg_size[0].append(t_avg_size / (last_idx-1-std_idx))
                dark_t_avg_change[0].append(total_change / change_cnt)
                dark_blink_rate[0].append(blink_cnt / ((last_idx- std_idx)/30))
                dark_close_rate[0].append(close_cnt / ((last_idx- std_idx)/30))
            elif condition == 'gamma':
                dark_t_avg_size[1].append(t_avg_size / (last_idx-1-std_idx))
                dark_t_avg_change[1].append(total_change / change_cnt)
                dark_blink_rate[1].append(blink_cnt / ((last_idx- std_idx)/30))
                dark_close_rate[1].append(close_cnt / ((last_idx- std_idx)/30))
            elif condition == 'temp':
                dark_t_avg_size[2].append(t_avg_size / (last_idx-1-std_idx))
                dark_t_avg_change[2].append(total_change / change_cnt)
                dark_blink_rate[2].append(blink_cnt / ((last_idx- std_idx)/30))
                dark_close_rate[2].append(close_cnt / ((last_idx- std_idx)/30))
            elif condition == 'bright':
                dark_t_avg_size[3].append(t_avg_size / (last_idx-1-std_idx))
                dark_t_avg_change[3].append(total_change / change_cnt)
                dark_blink_rate[3].append(blink_cnt / ((last_idx- std_idx)/30))
                dark_close_rate[3].append(close_cnt / ((last_idx- std_idx)/30))
            
        print('{} Done!! '.format(file_name))

s_csv_list_total.append(['명', '동공크기', '동공변화율', '깜빡임빈도', '감고있는시간'])
s_csv_list_total.append(['무자극',
                        sum(bright_t_avg_size[0])/len(bright_t_avg_size[0]),
                        sum(bright_t_avg_change[0])/len(bright_t_avg_change[0]),
                        sum(bright_blink_rate[0])/len(bright_blink_rate[0]),
                        sum(bright_close_rate[0])/len(bright_close_rate[0])])
s_csv_list_total.append(['감마변화',
                        sum(bright_t_avg_size[1])/len(bright_t_avg_size[1]),
                        sum(bright_t_avg_change[1])/len(bright_t_avg_change[1]),
                        sum(bright_blink_rate[1])/len(bright_blink_rate[1]),
                        sum(bright_close_rate[1])/len(bright_close_rate[1])])
s_csv_list_total.append(['색온도변화',
                        sum(bright_t_avg_size[2])/len(bright_t_avg_size[2]),
                        sum(bright_t_avg_change[2])/len(bright_t_avg_change[2]),
                        sum(bright_blink_rate[2])/len(bright_blink_rate[2]),
                        sum(bright_close_rate[2])/len(bright_close_rate[2])])
s_csv_list_total.append(['밝기변화',
                        sum(bright_t_avg_size[3])/len(bright_t_avg_size[3]),
                        sum(bright_t_avg_change[3])/len(bright_t_avg_change[3]),
                        sum(bright_blink_rate[3])/len(bright_blink_rate[3]),
                        sum(bright_close_rate[3])/len(bright_close_rate[3])])

s_csv_list_total.append(['암', '동공크기', '동공변화율', '깜빡임빈도', '감고있는시간'])
s_csv_list_total.append(['무자극',
                        sum(dark_t_avg_size[0])/len(dark_t_avg_size[0]),
                        sum(dark_t_avg_change[0])/len(dark_t_avg_change[0]),
                        sum(dark_blink_rate[0])/len(dark_blink_rate[0]),
                        sum(dark_close_rate[0])/len(dark_close_rate[0])])
s_csv_list_total.append(['감마변화',
                        sum(dark_t_avg_size[1])/len(dark_t_avg_size[1]),
                        sum(dark_t_avg_change[1])/len(dark_t_avg_change[1]),
                        sum(dark_blink_rate[1])/len(dark_blink_rate[1]),
                        sum(dark_close_rate[1])/len(dark_close_rate[1])])
s_csv_list_total.append(['색온도변화',
                        sum(dark_t_avg_size[2])/len(dark_t_avg_size[2]),
                        sum(dark_t_avg_change[2])/len(dark_t_avg_change[2]),
                        sum(dark_blink_rate[2])/len(dark_blink_rate[2]),
                        sum(dark_close_rate[2])/len(dark_close_rate[2])])
s_csv_list_total.append(['밝기변화',
                        sum(dark_t_avg_size[3])/len(dark_t_avg_size[3]),
                        sum(dark_t_avg_change[3])/len(dark_t_avg_change[3]),
                        sum(dark_blink_rate[3])/len(dark_blink_rate[3]),
                        sum(dark_close_rate[3])/len(dark_close_rate[3])])

s_csv_t = pd.DataFrame(s_csv_list_total, columns=csv_column_total)
csv_name_t = csv_save_path + 'total_3m.csv'
s_csv_t.to_csv(csv_name_t, encoding="euc-kr")
                              
print('total_3m_back.csv Done!! ')

01_bright_gamma Done!! 
01_bright_std Done!! 
01_bright_temp Done!! 
01_dark_bright Done!! 
01_dark_gamma Done!! 
01_dark_std Done!! 
01_dark_temp Done!! 
03_bright_bright Done!! 
03_bright_gamma Done!! 
03_bright_std Done!! 
03_bright_temp Done!! 
03_dark_bright Done!! 
03_dark_gamma Done!! 
03_dark_std Done!! 
03_dark_temp Done!! 
05_bright_bright Done!! 
05_bright_gamma Done!! 
05_bright_std Done!! 
05_bright_temp Done!! 
05_dark_bright Done!! 
05_dark_gamma Done!! 
05_dark_std Done!! 
05_dark_temp Done!! 
07_bright_bright Done!! 
07_bright_gamma Done!! 
07_bright_std Done!! 
07_bright_temp Done!! 
07_dark_bright Done!! 
07_dark_gamma Done!! 
07_dark_std Done!! 
07_dark_temp Done!! 
08_bright_bright Done!! 
08_bright_gamma Done!! 
08_bright_std Done!! 
08_bright_temp Done!! 
08_dark_bright Done!! 
08_dark_gamma Done!! 
08_dark_std Done!! 
08_dark_temp Done!! 
10_bright_bright Done!! 
10_bright_gamma Done!! 
10_bright_std Done!! 
10_bright_temp Done!! 
10_dark_bright Done!! 
10_dark_

[[], [], [], []]

In [3]:
csv_paths

['Z:/segmentation_pupil_v3/csv2\\02_dark_bright_1.csv',
 'Z:/segmentation_pupil_v3/csv2\\02_dark_bright_15.csv',
 'Z:/segmentation_pupil_v3/csv2\\02_dark_gamma_1.csv',
 'Z:/segmentation_pupil_v3/csv2\\02_dark_gamma_15.csv',
 'Z:/segmentation_pupil_v3/csv2\\02_dark_std_1.csv',
 'Z:/segmentation_pupil_v3/csv2\\02_dark_std_15.csv',
 'Z:/segmentation_pupil_v3/csv2\\02_dark_temp_1.csv',
 'Z:/segmentation_pupil_v3/csv2\\02_dark_temp_15.csv',
 'Z:/segmentation_pupil_v3/csv2\\04_dark_bright_1.csv',
 'Z:/segmentation_pupil_v3/csv2\\04_dark_bright_15.csv',
 'Z:/segmentation_pupil_v3/csv2\\04_dark_gamma_1.csv',
 'Z:/segmentation_pupil_v3/csv2\\04_dark_gamma_15.csv',
 'Z:/segmentation_pupil_v3/csv2\\04_dark_std_1.csv',
 'Z:/segmentation_pupil_v3/csv2\\04_dark_std_15.csv',
 'Z:/segmentation_pupil_v3/csv2\\04_dark_temp_1.csv',
 'Z:/segmentation_pupil_v3/csv2\\04_dark_temp_15.csv',
 'Z:/segmentation_pupil_v3/csv2\\06_dark_bright_1.csv',
 'Z:/segmentation_pupil_v3/csv2\\06_dark_bright_15.csv',
 'Z:/se

In [102]:
import csv
import glob
import jovian
import numpy
import pandas as pd

root_path = 'Z:/segmentation_pupil_v3/csv2/'
# root_path = 'C:/Users/th_k9/Desktop/analysis/result2/'
csv_save_path = './result2_3m/'


csv_paths = glob.glob(root_path + '*.csv')

csv_column = ['ori_Area', 'nom_Area', 'change_rate', 
              'Center_x', 'Center_y', 'Blink', 
              'Frame_num', 't_avg_size', 't_aver_change', 'c_aver_change', 'r_aver_change', 'Blink_cnt', 'Close_cnt', 'Blink_rate', 'Close_rate']
            # contraction, relaxation
csv_column_total = ['file_name', 'total_frame', 't_avg_size', 't_avg_change', 'c_avg_change',
                   'r_avg_change', 'blink_cnt', 'close_cnt', 'blink_rate', 'close_rate']
    
s_csv_list_total = []

# std, gamma, temp, bright
bright_t_avg_size = [[], [], [], []]
bright_t_avg_change = [[], [], [], []]
bright_blink_rate = [[], [], [], []]
bright_close_rate = [[], [], [], []]
dark_t_avg_size = [[], [], [], []]
dark_t_avg_change = [[], [], [], []]
dark_blink_rate = [[], [], [], []]
dark_close_rate = [[], [], [], []]
    
for csv_path in csv_paths:
    file_name = csv_path.split('\\')[-1][:-4]
    env_name = csv_path.split('\\')[-1].split('_')[1]
    condition = csv_path.split('\\')[-1].split('_')[2]
    
    csv = pd.read_csv(csv_path, error_bad_lines=False)
    ori_csv = list(csv['ori_Area'])
    
    if file_name.split('_')[-1] == '1':
        mean_bright = 0
        for idx in range(csv.shape[0]):
            mean_bright += csv['ori_Area'][idx]
        mean_bright /= csv.shape[0]
    else:
        mean_bright = 0
        
        std_idx = csv.shape[0] - 5401
        last_idx = csv.shape[0] - 1
#         std_idx = 0
#         last_idx = 5400
        
        s_csv_list = []
        t_avg_size = 0
        csv['ori_Area'] -= mean_bright
        
        blink = False
        blink_cnt = 0
        close_cnt = 0
        bf_lists = []
        m_change = []
        p_change = []
        change_cnt = 0
        total_change = 0
        
        ff_cnt = 0
        for idx in range(std_idx, last_idx):
            t_avg_size += ori_csv[idx]
            
            c_x = 0
            c_y = 0
             
            change_rate = ''
            if blink == False:
                if csv['ori_Area'][idx] == -mean_bright and idx != std_idx:
                    blink = True
                    bf_list = [idx]
                    blink_start_idx = idx
                    blink_cnt += 1  
                    
                    before_area = csv['ori_Area'][idx - 1]
        
                derivative = ori_csv[idx + 1] - ori_csv[idx]
                
                if idx == std_idx:
                    s_idx = idx
                    if derivative >= 0:
                        sign_check = True
                    else:
                        sign_check = False
                else:
                    change_f = -1
                    change_a = 0
                    if sign_check == True:
                        if derivative < 0:
                            change_f = idx - s_idx
                            change_a = ori_csv[idx] - ori_csv[s_idx]
                            sign_check = False
                            s_idx = idx
                            
                    else:
                        if derivative > 0:
                            change_f = idx - s_idx
                            change_a = ori_csv[idx] - ori_csv[s_idx]
                            sign_check = True
                            s_idx = idx
                    
                    if change_f != -1:
                        change_cnt += 1
                        
                        change_rate = change_a / change_f
                        
                        total_change += abs(change_rate)
                        
                        if change_rate > 0 :
                            p_change.append(change_rate)
                        else:
                            m_change.append(change_rate)
                
                if blink == True:
                    row = [ori_csv[idx], csv['ori_Area'][idx], change_rate, c_x, c_y, int(blink)]
                else:
                    row = [ori_csv[idx], csv['ori_Area'][idx], change_rate, c_x, c_y, int(blink)]
            else:
                derivative = 0
                
                row = [ori_csv[idx], csv['ori_Area'][idx], change_rate, c_x, c_y, int(blink)]
            s_csv_list.append(row)
            
            if blink == True:
                close_cnt += 1
                if idx < last_idx - 1:
                    diff = csv['ori_Area'][idx+1] - csv['ori_Area'][idx] 
                    if diff < 0:
                        blink = False
                        bf_list.append(idx)
                        bf_lists.append(bf_list)
                        
                if idx == last_idx - 1:
                    bf_list.append(idx)
                    bf_lists.append(bf_list)
                    
            ff_cnt += 1
        
        s_csv_list[0].append(last_idx-std_idx)
        s_csv_list[0].append(t_avg_size / (last_idx -1- std_idx))
        s_csv_list[0].append(total_change / change_cnt)
        s_csv_list[0].append(sum(m_change) / len(m_change))
        s_csv_list[0].append(sum(p_change) / len(p_change))
        s_csv_list[0].append(blink_cnt)
        s_csv_list[0].append(close_cnt)
        s_csv_list[0].append(blink_cnt / ((last_idx - std_idx)/30))
        s_csv_list[0].append(close_cnt / ((last_idx - std_idx)/30))
        s_csv = pd.DataFrame(s_csv_list, columns=csv_column)
        csv_name = csv_save_path + file_name + '.csv'
        s_csv.to_csv(csv_name)
        
        file_name = csv_path.split('\\')[-1][:-7]
        s_csv_list_total.append(
                            [file_name, last_idx - std_idx, t_avg_size / (last_idx-1-std_idx), total_change / change_cnt,
                           sum(m_change) / len(m_change), sum(p_change) / len(p_change),
                            blink_cnt, close_cnt, blink_cnt / ((last_idx- std_idx)/30), close_cnt / ((last_idx- std_idx)/30)]
                            )
        
        # std, gamma, temp, bright
        if env_name == 'bright':
            if condition == 'std':
                bright_t_avg_size[0].append(t_avg_size / (last_idx - std_idx))
                bright_t_avg_change[0].append(total_change / change_cnt)
                bright_blink_rate[0].append(blink_cnt / ((last_idx - std_idx)/30))
                bright_close_rate[0].append(close_cnt / ((last_idx - std_idx)/30))
            elif condition == 'gamma':
                bright_t_avg_size[1].append(t_avg_size / (last_idx-1-std_idx))
                bright_t_avg_change[1].append(total_change / change_cnt)
                bright_blink_rate[1].append(blink_cnt / ((last_idx- std_idx)/30))
                bright_close_rate[1].append(close_cnt / ((last_idx- std_idx)/30))
            elif condition == 'temp':
                bright_t_avg_size[2].append(t_avg_size / (last_idx-1-std_idx))
                bright_t_avg_change[2].append(total_change / change_cnt)
                bright_blink_rate[2].append(blink_cnt / ((last_idx- std_idx)/30))
                bright_close_rate[2].append(close_cnt / ((last_idx- std_idx)/30))
            elif condition == 'bright':
                bright_t_avg_size[3].append(t_avg_size / (last_idx-1-std_idx))
                bright_t_avg_change[3].append(total_change / change_cnt)
                bright_blink_rate[3].append(blink_cnt / ((last_idx- std_idx)/30))
                bright_close_rate[3].append(close_cnt / ((last_idx- std_idx)/30))

        elif env_name == 'dark':
            if condition == 'std':
                dark_t_avg_size[0].append(t_avg_size / (last_idx-1-std_idx))
                dark_t_avg_change[0].append(total_change / change_cnt)
                dark_blink_rate[0].append(blink_cnt / ((last_idx- std_idx)/30))
                dark_close_rate[0].append(close_cnt / ((last_idx- std_idx)/30))
            elif condition == 'gamma':
                dark_t_avg_size[1].append(t_avg_size / (last_idx-1-std_idx))
                dark_t_avg_change[1].append(total_change / change_cnt)
                dark_blink_rate[1].append(blink_cnt / ((last_idx- std_idx)/30))
                dark_close_rate[1].append(close_cnt / ((last_idx- std_idx)/30))
            elif condition == 'temp':
                dark_t_avg_size[2].append(t_avg_size / (last_idx-1-std_idx))
                dark_t_avg_change[2].append(total_change / change_cnt)
                dark_blink_rate[2].append(blink_cnt / ((last_idx- std_idx)/30))
                dark_close_rate[2].append(close_cnt / ((last_idx- std_idx)/30))
            elif condition == 'bright':
                dark_t_avg_size[3].append(t_avg_size / (last_idx-1-std_idx))
                dark_t_avg_change[3].append(total_change / change_cnt)
                dark_blink_rate[3].append(blink_cnt / ((last_idx- std_idx)/30))
                dark_close_rate[3].append(close_cnt / ((last_idx- std_idx)/30))
            
        print('{} Done!! '.format(file_name))

s_csv_list_total.append(['명', '동공크기', '동공변화율', '깜빡임빈도', '감고있는시간'])
s_csv_list_total.append(['무자극',
                        sum(bright_t_avg_size[0])/len(bright_t_avg_size[0]),
                        sum(bright_t_avg_change[0])/len(bright_t_avg_change[0]),
                        sum(bright_blink_rate[0])/len(bright_blink_rate[0]),
                        sum(bright_close_rate[0])/len(bright_close_rate[0])])
s_csv_list_total.append(['감마변화',
                        sum(bright_t_avg_size[1])/len(bright_t_avg_size[1]),
                        sum(bright_t_avg_change[1])/len(bright_t_avg_change[1]),
                        sum(bright_blink_rate[1])/len(bright_blink_rate[1]),
                        sum(bright_close_rate[1])/len(bright_close_rate[1])])
s_csv_list_total.append(['색온도변화',
                        sum(bright_t_avg_size[2])/len(bright_t_avg_size[2]),
                        sum(bright_t_avg_change[2])/len(bright_t_avg_change[2]),
                        sum(bright_blink_rate[2])/len(bright_blink_rate[2]),
                        sum(bright_close_rate[2])/len(bright_close_rate[2])])
s_csv_list_total.append(['밝기변화',
                        sum(bright_t_avg_size[3])/len(bright_t_avg_size[3]),
                        sum(bright_t_avg_change[3])/len(bright_t_avg_change[3]),
                        sum(bright_blink_rate[3])/len(bright_blink_rate[3]),
                        sum(bright_close_rate[3])/len(bright_close_rate[3])])

s_csv_list_total.append(['암', '동공크기', '동공변화율', '깜빡임빈도', '감고있는시간'])
s_csv_list_total.append(['무자극',
                        sum(dark_t_avg_size[0])/len(dark_t_avg_size[0]),
                        sum(dark_t_avg_change[0])/len(dark_t_avg_change[0]),
                        sum(dark_blink_rate[0])/len(dark_blink_rate[0]),
                        sum(dark_close_rate[0])/len(dark_close_rate[0])])
s_csv_list_total.append(['감마변화',
                        sum(dark_t_avg_size[1])/len(dark_t_avg_size[1]),
                        sum(dark_t_avg_change[1])/len(dark_t_avg_change[1]),
                        sum(dark_blink_rate[1])/len(dark_blink_rate[1]),
                        sum(dark_close_rate[1])/len(dark_close_rate[1])])
s_csv_list_total.append(['색온도변화',
                        sum(dark_t_avg_size[2])/len(dark_t_avg_size[2]),
                        sum(dark_t_avg_change[2])/len(dark_t_avg_change[2]),
                        sum(dark_blink_rate[2])/len(dark_blink_rate[2]),
                        sum(dark_close_rate[2])/len(dark_close_rate[2])])
s_csv_list_total.append(['밝기변화',
                        sum(dark_t_avg_size[3])/len(dark_t_avg_size[3]),
                        sum(dark_t_avg_change[3])/len(dark_t_avg_change[3]),
                        sum(dark_blink_rate[3])/len(dark_blink_rate[3]),
                        sum(dark_close_rate[3])/len(dark_close_rate[3])])

s_csv_t = pd.DataFrame(s_csv_list_total, columns=csv_column_total)
csv_name_t = csv_save_path + 'total_3m_back.csv'
s_csv_t.to_csv(csv_name_t, encoding="euc-kr")
                              
print('total_3m_back.csv Done!! ')

(0, 5400)

In [47]:
len(ori_csv), idx, s_idx

(27003, 21603, 27011)

In [17]:
import glob
import pandas as pd

ot_path = 'Z:/segmentation_pupil_v3/csv/'
csv_paths = glob.glob(root_path + '*.csv')
csv = pd.read_csv(csv_paths[3], error_bad_lines=False)

file_name = csv_paths[3].split('\\')[-1][:-7]

file_name

'01_bright_std'

# 1차 완성

In [44]:
import csv
import glob
import jovian
import numpy
import pandas as pd

root_path = 'Z:/segmentation_pupil_v3/csv/'
csv_save_path = './result2/'

csv_paths = glob.glob(root_path + '*.csv')
csv_paths = csv_paths[211:]

csv_column = ['ori_Area', 'nom_Area', 'change_rate', 
              'Center_x', 'Center_y', 'Blink', 
              'Frame_num', 't_avg_size', 't_aver_change', 'c_aver_change', 'r_aver_change', 'Blink_cnt', 'Blink_rate', 'Close_cnt', 'Close_rate']
            # contraction, relaxation
for csv_path in csv_paths:
    file_name = csv_path.split('\\')[-1][:-4]
    
    csv = pd.read_csv(csv_path, error_bad_lines=False)
    ori_csv = list(csv['Area'])
    
    if file_name.split('_')[-1] == '1':
        mean_bright = 0
        for idx in range(csv.shape[0]):
            mean_bright += csv['Area'][idx]
        mean_bright /= csv.shape[0]
    else:
        s_csv_list = []
        t_avg_size = 0
        csv['Area'] -= mean_bright
        
        blink = False
        blink_cnt = 0
        close_cnt = 0
        bf_lists = []
        m_change = []
        p_change = []
        change_cnt = 0
        total_change = 0
        for idx in range(csv.shape[0] - 1):
            t_avg_size += ori_csv[idx]
            
            c_x = csv['Center'][idx].strip('()').split(',')[0]
            c_y = csv['Center'][idx].strip('()').split(',')[1].lstrip()
             
            change_rate = ''
            if blink == False:
                if csv['Area'][idx] == -mean_bright and idx != 0:
                    blink = True
                    bf_list = [idx]
                    blink_start_idx = idx
                    blink_cnt += 1  
                    
                    before_area = csv['Area'][idx - 1]
        
                derivative = ori_csv[idx + 1] - ori_csv[idx]
                
                if idx == 0:
                    s_idx = idx
                    if derivative >= 0:
                        sign_check = True
                    else:
                        sign_check = False
                else:
                    change_f = -1
                    change_a = 0
                    if sign_check == True:
                        if derivative < 0:
                            change_f = idx - s_idx
                            change_a = ori_csv[idx] - ori_csv[s_idx]
                            sign_check = False
                            s_idx = idx
                            
                    else:
                        if derivative > 0:
                            change_f = idx - s_idx
                            change_a = ori_csv[idx] - ori_csv[s_idx]
                            sign_check = True
                            s_idx = idx
                    
                    if change_f != -1:
                        change_cnt += 1
                        
                        change_rate = change_a / change_f
                        
                        total_change += abs(change_rate)
                        
                        if change_rate > 0 :
                            p_change.append(change_rate)
                        else:
                            m_change.append(change_rate)
                
                if blink == True:
                    row = [ori_csv[idx], csv['Area'][idx], change_rate, c_x, c_y, int(blink)]
                else:
                    row = [ori_csv[idx], csv['Area'][idx], change_rate, c_x, c_y, int(blink)]
            else:
                derivative = 0
                
                row = [ori_csv[idx], csv['Area'][idx], change_rate, c_x, c_y, int(blink)]
            s_csv_list.append(row)
            
            if blink == True:
                close_cnt += 1
                if idx < csv.shape[0] - 1:
                    diff = csv['Area'][idx+1] - csv['Area'][idx] 
                    if diff < 0:
                        blink = False
                        bf_list.append(idx)
                        bf_lists.append(bf_list)
                        
                if idx == csv.shape[0] - 1:
                    bf_list.append(idx)
                    bf_lists.append(bf_list)
        
        s_csv_list[0].append(csv.shape[0])
        s_csv_list[0].append(t_avg_size / (csv.shape[0]-1))
        s_csv_list[0].append(total_change / change_cnt)
        s_csv_list[0].append(sum(m_change) / len(m_change))
        s_csv_list[0].append(sum(p_change) / len(p_change))
        s_csv_list[0].append(blink_cnt)
        s_csv_list[0].append(blink_cnt / (csv.shape[0]/30))
        s_csv_list[0].append(close_cnt)
        s_csv_list[0].append(close_cnt / (csv.shape[0]/30))
        s_csv = pd.DataFrame(s_csv_list, columns=csv_column)
        csv_name = csv_save_path + file_name + '.csv'
        s_csv.to_csv(csv_name)
        
        print('{} Done!! '.format(file_name))

17_bright_bright_15 Done!! 
17_bright_gamma_15 Done!! 
17_bright_std_15 Done!! 
17_bright_temp_15 Done!! 
17_dark_bright_15 Done!! 
17_dark_gmma_15 Done!! 
17_dark_std_15 Done!! 
17_dark_temp_15 Done!! 


In [43]:
csv_paths = glob.glob(root_path + '*.csv')
csv_paths = csv_paths[211:]
csv_paths

['Z:/segmentation_pupil_v3/csv\\17_bright_bright_1.csv',
 'Z:/segmentation_pupil_v3/csv\\17_bright_bright_15.csv',
 'Z:/segmentation_pupil_v3/csv\\17_bright_gamma_1.csv',
 'Z:/segmentation_pupil_v3/csv\\17_bright_gamma_15.csv',
 'Z:/segmentation_pupil_v3/csv\\17_bright_std_1.csv',
 'Z:/segmentation_pupil_v3/csv\\17_bright_std_15.csv',
 'Z:/segmentation_pupil_v3/csv\\17_bright_temp_1.csv',
 'Z:/segmentation_pupil_v3/csv\\17_bright_temp_15.csv',
 'Z:/segmentation_pupil_v3/csv\\17_dark_bright_1.csv',
 'Z:/segmentation_pupil_v3/csv\\17_dark_bright_15.csv',
 'Z:/segmentation_pupil_v3/csv\\17_dark_gamma_1.csv',
 'Z:/segmentation_pupil_v3/csv\\17_dark_gmma_15.csv',
 'Z:/segmentation_pupil_v3/csv\\17_dark_std_1.csv',
 'Z:/segmentation_pupil_v3/csv\\17_dark_std_15.csv',
 'Z:/segmentation_pupil_v3/csv\\17_dark_temp_1.csv',
 'Z:/segmentation_pupil_v3/csv\\17_dark_temp_15.csv']

# Version 4

In [69]:
import csv
import glob
import jovian
import numpy
import pandas as pd

root_path = 'E:/segmentation_pupil/'

csv_paths = glob.glob(root_path + '*/*.csv')
csv_paths = csv_paths[:16]

csv_column = ['ori_Area', 'nom_Area', 'remove_blink', 'derivative',  
              'change_rate', 'Center_x', 'Center_y', 'Blink', 
              't_aver_change', 'c_aver_change', 'r_aver_change', 'Blink_cnt', 'Blink_rate', 'Close_cnt', 'Close_rate']
            # contraction, relaxation
for csv_path in csv_paths:
    file_name = csv_path.split('\\')[-1][:-4]
    
    csv = pd.read_csv(csv_path, error_bad_lines=False)
    ori_csv = list(csv['Area'])
    
    if file_name.split('_')[-1] == '1':
        mean_bright = 0
        for idx in range(csv.shape[0]):
            mean_bright += csv['Area'][idx]
        mean_bright /= csv.shape[0]
    else:
        s_csv_list = [] 
        csv['Area'] -= mean_bright
        
        blink = False
        blink_cnt = 0
        close_cnt = 0
        bf_lists = []
        m_change = []
        p_change = []
        change_cnt = 0
        total_change = 0
        for idx in range(csv.shape[0] - 1):
            c_x = csv['Center'][idx].strip('()').split(',')[0]
            c_y = csv['Center'][idx].strip('()').split(',')[1].lstrip()
             
            change_rate = ''
            if blink == False:
                if csv['Area'][idx] == -mean_bright:
                    blink = True
                    bf_list = [idx]
                    blink_start_idx = idx
                    blink_cnt += 1  
                    
                    before_area = csv['Area'][idx - 1]
        
                derivative = ori_csv[idx + 1] - ori_csv[idx]
                
                if idx == 0:
                    s_idx = idx
                    if derivative >= 0:
                        sign_check = True
                    else:
                        sign_check = False
                else:
                    change_f = -1
                    change_a = 0
                    if sign_check == True:
                        if derivative < 0:
                            change_f = idx - s_idx
                            change_a = ori_csv[idx] - ori_csv[s_idx]
                            sign_check = False
                            s_idx = idx
                            
                    else:
                        if derivative > 0:
                            change_f = idx - s_idx
                            change_a = ori_csv[idx] - ori_csv[s_idx]
                            sign_check = True
                            s_idx = idx
                    
                    if change_f != -1:
                        change_cnt += 1
                        
                        change_rate = change_a / change_f
                        
                        total_change += abs(change_rate)
                        
                        if change_rate > 0 :
                            p_change.append(change_rate)
                        else:
                            m_change.append(change_rate)
                
                if blink == True:
                    row = [ori_csv[idx], csv['Area'][idx], before_area, derivative, change_rate, c_x, c_y, int(blink)]
                else:
                    row = [ori_csv[idx], csv['Area'][idx], csv['Area'][idx], derivative, change_rate, c_x, c_y, int(blink)]
            else:
                derivative = 0
                
                row = [ori_csv[idx], csv['Area'][idx], before_area, derivative,  change_rate, c_x, c_y, int(blink)]
            s_csv_list.append(row)
            
            if blink == True:
                close_cnt += 1
                if idx < csv.shape[0] - 1:
                    diff = csv['Area'][idx+1] - csv['Area'][idx] 
                    if diff < 0:
                        blink = False

        
        s_csv_list[0].append(total_change / change_cnt)
        s_csv_list[0].append(sum(m_change) / len(m_change))
        s_csv_list[0].append(sum(p_change) / len(p_change))
        s_csv_list[0].append(blink_cnt)
        s_csv_list[0].append(blink_cnt / csv.shape[0])
        s_csv_list[0].append(close_cnt)
        s_csv_list[0].append(close_cnt / csv.shape[0])
        s_csv = pd.DataFrame(s_csv_list, columns=csv_column)
        csv_name = './' + file_name + '.csv'
        s_csv.to_csv(csv_name)

b'Skipping line 260: expected 3 fields, saw 5\nSkipping line 1583: expected 3 fields, saw 5\nSkipping line 6147: expected 3 fields, saw 5\n'
b'Skipping line 9: expected 3 fields, saw 7\nSkipping line 142: expected 3 fields, saw 5\n'
b'Skipping line 10794: expected 3 fields, saw 5\nSkipping line 11584: expected 3 fields, saw 5\nSkipping line 14503: expected 3 fields, saw 5\nSkipping line 19828: expected 3 fields, saw 5\nSkipping line 20270: expected 3 fields, saw 5\n'
b'Skipping line 2352: expected 3 fields, saw 5\nSkipping line 2895: expected 3 fields, saw 5\nSkipping line 7709: expected 3 fields, saw 5\nSkipping line 9006: expected 3 fields, saw 5\nSkipping line 9892: expected 3 fields, saw 5\nSkipping line 15145: expected 3 fields, saw 5\nSkipping line 15261: expected 3 fields, saw 5\nSkipping line 15601: expected 3 fields, saw 5\nSkipping line 15828: expected 3 fields, saw 5\nSkipping line 15853: expected 3 fields, saw 5\nSkipping line 15893: expected 3 fields, saw 5\nSkipping line 

# Version 3

In [64]:
import csv
import glob
import jovian
import numpy
import pandas as pd

root_path = 'E:/segmentation_pupil/'

csv_paths = glob.glob(root_path + '*/*.csv')
csv_paths = csv_paths[:16]

csv_column = ['ori_Area', 'remove_blink', 'derivative',  'change_rate', 
              'nom_Area', 'Center_x', 'Center_y', 'Blink', 
              't_aver_change', 'c_aver_change', 'r_aver_change', 'Blink_cnt', 'Blink_rate', 'Close_cnt', 'Close_rate']
            # contraction, relaxation
for csv_path in csv_paths:
    file_name = csv_path.split('\\')[-1][:-4]
    
    csv = pd.read_csv(csv_path, error_bad_lines=False)
    ori_csv = list(csv['Area'])
    
    if file_name.split('_')[-1] == '1':
        mean_bright = 0
        for idx in range(csv.shape[0]):
            mean_bright += csv['Area'][idx]
        mean_bright /= csv.shape[0]
    else:
        s_csv_list = [] 
        csv['Area'] -= mean_bright
        
        blink = False
        blink_cnt = 0
        close_cnt = 0
        bf_lists = []
        m_change = []
        p_change = []
        change_cnt = 0
        total_change = 0
        for idx in range(csv.shape[0] - 1):
            c_x = csv['Center'][idx].strip('()').split(',')[0]
            c_y = csv['Center'][idx].strip('()').split(',')[1].lstrip()
             
            change_rate = ''
            if blink == False:
                if csv['Area'][idx] == -mean_bright:
                    blink = True
                    bf_list = [idx]
                    blink_start_idx = idx
                    blink_cnt += 1  
                    
                    before_area = ori_csv[idx-1]
        
                derivative = ori_csv[idx + 1] - ori_csv[idx]
                
                if idx == 0:
                    s_idx = idx
                    if derivative >= 0:
                        sign_check = True
                    else:
                        sign_check = False
                else:
                    change_f = -1
                    change_a = 0
                    if sign_check == True:
                        if derivative < 0:
                            change_f = idx - s_idx
                            change_a = ori_csv[idx] - ori_csv[s_idx]
                            sign_check = False
                            s_idx = idx
                            
                    else:
                        if derivative > 0:
                            change_f = idx - s_idx
                            change_a = ori_csv[idx] - ori_csv[s_idx]
                            sign_check = True
                            s_idx = idx
                    
                    if change_f != -1:
                        change_cnt += 1
                        
                        change_rate = change_a / change_f
                        
                        total_change += abs(change_rate)
                        
                        if change_rate > 0 :
                            p_change.append(change_rate)
                        else:
                            m_change.append(change_rate)
                        
                row = [ori_csv[idx], ori_csv[idx], derivative, change_rate, csv['Area'][idx], c_x, c_y, int(blink)]
            else:
                derivative = 0
                
                row = [ori_csv[idx], before_area, derivative,  change_rate, csv['Area'][idx], c_x, c_y, int(blink)]
            s_csv_list.append(row)
            
            if blink == True:
                close_cnt += 1
                if idx < csv.shape[0] - 1:
                    diff = csv['Area'][idx+1] - csv['Area'][idx] 
                    if diff < 0:
                        blink = False
                        bf_list.append(idx)
                        bf_lists.append(bf_list)
                        
                if idx == csv.shape[0] - 1:
                    bf_list.append(idx)
                    bf_lists.append(bf_list)
        
        s_csv_list[0].append(total_change / change_cnt)
        s_csv_list[0].append(sum(m_change) / len(m_change))
        s_csv_list[0].append(sum(p_change) / len(p_change))
        s_csv_list[0].append(blink_cnt)
        s_csv_list[0].append(blink_cnt / csv.shape[0])
        s_csv_list[0].append(close_cnt)
        s_csv_list[0].append(close_cnt / csv.shape[0])
        s_csv = pd.DataFrame(s_csv_list, columns=csv_column)
        csv_name = './' + file_name + '.csv'
        s_csv.to_csv(csv_name)

b'Skipping line 260: expected 3 fields, saw 5\nSkipping line 1583: expected 3 fields, saw 5\nSkipping line 6147: expected 3 fields, saw 5\n'
b'Skipping line 9: expected 3 fields, saw 7\nSkipping line 142: expected 3 fields, saw 5\n'
b'Skipping line 10794: expected 3 fields, saw 5\nSkipping line 11584: expected 3 fields, saw 5\nSkipping line 14503: expected 3 fields, saw 5\nSkipping line 19828: expected 3 fields, saw 5\nSkipping line 20270: expected 3 fields, saw 5\n'
b'Skipping line 2352: expected 3 fields, saw 5\nSkipping line 2895: expected 3 fields, saw 5\nSkipping line 7709: expected 3 fields, saw 5\nSkipping line 9006: expected 3 fields, saw 5\nSkipping line 9892: expected 3 fields, saw 5\nSkipping line 15145: expected 3 fields, saw 5\nSkipping line 15261: expected 3 fields, saw 5\nSkipping line 15601: expected 3 fields, saw 5\nSkipping line 15828: expected 3 fields, saw 5\nSkipping line 15853: expected 3 fields, saw 5\nSkipping line 15893: expected 3 fields, saw 5\nSkipping line 

# Version 2

In [18]:
import csv
import glob
import jovian
import numpy
import pandas as pd

root_path = 'E:/segmentation_pupil/'

csv_paths = glob.glob(root_path + '*/*.csv')
csv_paths = csv_paths[:2]

for csv_path in csv_paths:
    file_name = csv_path.split('\\')[-1][:-4]

    csv = pd.read_csv(csv_path)
    ori_csv = list(csv['Area'])
    
    if file_name.split('_')[-1] == '1':
        mean_bright = 0
        for idx in range(csv.shape[0]):
            mean_bright += csv['Area'][idx]
        mean_bright /= csv.shape[0]
    else:
        s_csv_list = [] 
        csv['Area'] -= mean_bright
        
        blink = False
        blink_cnt = 0
        close_cnt = 0
        bf_lists = []
        for idx in range(csv.shape[0]):
            c_x = csv['Center'][idx].strip('()').split(',')[0]
            c_y = csv['Center'][idx].strip('()').split(',')[1].lstrip()
            
            if blink == False:
                if csv['Area'][idx] == -mean_bright:
                    blink = True
                    bf_list = [idx]
                    blink_start_idx = idx
                    blink_cnt += 1  
                    
                    before_area = ori_csv[idx-1]
                    
            if blink == False:
                s_csv_list.append([ori_csv[idx], ori_csv[idx], csv['Area'][idx], c_x, c_y, int(blink)])
            else:
                s_csv_list.append([ori_csv[idx], before_area, csv['Area'][idx], c_x, c_y, int(blink)])
            
            if blink == True:
                close_cnt += 1
                if idx < csv.shape[0] - 1:
                    diff = csv['Area'][idx+1] - csv['Area'][idx] 
                    if diff < 0:
                        blink = False
                        bf_list.append(idx)
                        bf_lists.append(bf_list)
                        
                if idx == csv.shape[0] - 1:
                    bf_list.append(idx)
                    bf_lists.append(bf_list)
        
        s_csv_list[0].append(blink_cnt)
        s_csv_list[0].append(blink_cnt / csv.shape[0])
        s_csv_list[0].append(close_cnt)
        s_csv_list[0].append(close_cnt / csv.shape[0])
        s_csv = pd.DataFrame(s_csv_list, columns=['ori_Area', 'remove_blink', 'nom_Area', 'Center_x', 'Center_y', 'Blink', 'Blink_cnt', 'Blink_rate', 'Close_cnt', 'Close_rate'])
        s_csv.to_csv('./result.csv')

# Version 1

In [ ]:
import csv
import glob
import jovian
import numpy
import pandas as pd

root_path = 'E:/segmentation_pupil/'

csv_paths = glob.glob(root_path + '*/*.csv')
csv_paths = csv_paths[:2]

for csv_path in csv_paths:
    file_name = csv_path.split('\\')[-1][:-4]

    csv = pd.read_csv(csv_path)
    
    if file_name.split('_')[-1] == '1':
        mean_bright = 0
        for idx in range(csv.shape[0]):
            mean_bright += csv['Area'][idx]
        mean_bright /= csv.shape[0]
    else:
        s_csv_list = []
        
        csv['Area'] -= mean_bright
        
        blink = False
        blink_cnt = 0
        close_cnt = 0
        bf_lists = []
        for idx in range(csv.shape[0]):
            c_x = csv['Center'][idx].strip('()').split(',')[0]
            c_y = csv['Center'][idx].strip('()').split(',')[1].lstrip()
            
            if blink == False:
                if csv['Area'][idx] == -mean_bright:
                    blink = True
                    bf_list = [idx]
                    blink_start_idx = idx
                    blink_cnt += 1
                
            s_csv_list.append([csv['Area'][idx], c_x, c_y, int(blink)])
                
            if blink == True:
                close_cnt += 1
                if idx < csv.shape[0] - 1:
                    diff = csv['Area'][idx+1] - csv['Area'][idx] 
                    if diff < 0:
                        blink = False
                        bf_list.append(idx)
                        bf_lists.append(bf_list)
                        
                if idx == csv.shape[0] - 1:
                    bf_list.append(idx)
                    bf_lists.append(bf_list)
        
        s_csv_list[0].append(blink_cnt)
        s_csv_list[0].append(blink_cnt / csv.shape[0])
        s_csv_list[0].append(close_cnt)
        s_csv_list[0].append(close_cnt / csv.shape[0])
        s_csv = pd.DataFrame(s_csv_list, columns=['Area', 'Center_x', 'Center_y', 'Blink', 'Blink_cnt', 'Blink_rate', 'Close_cnt', 'Close_rate'])
        s_csv.to_csv('./result.csv')